#  Organización de datos en lotes de entrenamiento

El siguiente paso es construir los lotes de entrenamiento de manera efectiva. Esto implica definir un método que garantice que el modelo reciba los datos de entrenamiento formateados durante el proceso de ajuste.

![Texto alternativo](./imgs/7.5.png)

En la sección 06,   los  lotes  de  entrenamiento  fueron  creados  automáticamente  por  la  clase  DataLoader  de  PyTorch ,  que  emplea  una  función  de  intercalación  predeterminada  para  combinar  listas  de  muestras  en  lotes.  Esta  función  se  encarga  de  tomar  una  lista  de muestras  de  datos  individuales  y  combinarlas  en  un  único  lote  que  el  modelo  puede  procesar  eficientemente  durante  el  entrenamiento.

Sin  embargo,  el  proceso  de  procesamiento  por  lotes  para  el  ajuste  fino  de  instrucciones  en  esta sección es  un  poco  más  complejo  y  requiere  la  creación  de  una  función  de intercalación  personalizada  que  posteriormente se conectará  al  DataLoader.

En  esta  sección,  se abordará  el  proceso  de  procesamiento  por  lotes  en  varios  pasos,  incluida  la  codificación de  la  función  de  intercalación  personalizada.

![Texto alternativo](./imgs/7.6.png)

Primero se implementan los pasos 2.1 y 2.2, se codifica una clase InstructionDataset que aplica format_input y pre_tokeniza todas las entradas en el conjunto de datos.

![Texto alternativo](./imgs/7.7.png)


In [1]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    return instruction_text + input_text

In [2]:
import torch
from torch.utils.data import Dataset

class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.encoded_text = []

        for entry in data:                                       
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

    def __getitem__(self, index):
        return self.encoded_text[index]
    
    def __len__(self):
        return len(self.data)

Similar al enfoque de la sección 06, se busca acelerar el entrenamiento recopilando múltiples ejemplos de entrenamiento en un lote, lo que requiere rellenar todas las entradas con una logitud similar.  En  lugar  de  añadir  los  tokens  <|endoftext|>  a  las  entradas  de  texto,  se puede añadir  su  ID  de  token  directamente  a  las  entradas  pretokenizadas.  

En  la sección 06, se igualaba  la  longitud  de  todos  los  ejemplos  de  un  conjunto  de  datos.  Pasando  al  paso  2.3  de  la  figura, se adopta  un  enfoque  más  sofisticado:  desarrollar  una  función  de  intercalación  personalizada  que  se puede pasar  al  cargador  de  datos.  Esta  función  de  intercalación  personalizada  iguala  la  longitud  de  los  ejemplos  de  entrenamiento  de  cada  lote,  permitiendo  que  los  distintos  lotes  tengan  longitudes  diferentes.  Este  enfoque  minimiza  el  relleno  innecesario,  extendiendo  las  secuencias  solo  para  que  coincidan  con  la  más  larga  de  cada  lote,  no  con  todo  el  conjunto  de  datos.

![Texto alternativo](./imgs/7.8.png)

In [3]:
def custom_collate_draft_1(batch, pad_token_id=50256, device='cpu'):
    #Calculamos las longitudes originales de cada secuencia
    batchs_lengths = [len(item) for item in batch]
    #Determinamos la longitud máxima del batch (+1 para añadir el token final)
    batch_max_length = max(batchs_lengths) + 1
    inputs_lst = []

    for item in batch:
        item += [pad_token_id]
        padded = item + [pad_token_id] * (batch_max_length - len(item))
        #Creamos el tensor de entrada quitando el último token ([:-1])
        #Esto se hace para alinear inputs y labels en el entrenamiento autoregresivo
        #El modelo ve "inputs" y debe predecir el siguiente token (label)
        #Por eso inputs = secuencia sin el último token
        #y labels  = secuencia sin el primero
        input = torch.tensor(padded[:-1])
        inputs_lst.append(input)
    
    inputs_tensor = torch.stack(inputs_lst).to(device)
    return inputs_tensor

inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

print(custom_collate_draft_1(batch))

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])


Como se puede  ver  en  base  al  resultado  anterior,  todas  las  entradas  se  han  rellenado  hasta  la  longitud  dela  lista  de  entrada  más  larga,  inputs_1,  que  contiene  5  ID  de  token. Sin  embargo,  como  se vio en las secciones 05  y  06,  también  se necesitan  crear  lotes con  los  ID  de  token  de  destino,  correspondientes  al  lote  de  ID  de  entrada.  Estos  ID  de  destino, son  cruciales  porque  representan  lo  que quiere  que  el  modelo  genera y  lo  que  se necesita  durante  el  entrenamiento  para  calcular  la  pérdida  para  las  actualizaciones  de  peso.

![Texto alternativo](./imgs/7.9.png)


Similar  al  proceso  descrito  en la sección 05  para  el  preentrenamiento  de  un  LLM,  los  ID  de  los  tokens  de  destino  coinciden  con  los  de  entrada,  pero  se  desplazan  una  posición  a  la  derecha.  Esta  configuración,  como  se  muestra  en  la  figura,  permite  al  LLM  aprender  a  predecir  el  siguiente  token  en  una  secuencia.

![Texto alternativo](./imgs/7.10.png)

La  siguiente  función  de  intercalación  actualizada  genera  los  ID  de  token  de  destino,  a  partir  de  los  ID  de  token  de  entrada:

In [4]:
def custom_collate_draft_2(batch,pad_token_id=50256,device="cpu"):
    batchs_lengths = [len(item) for item in batch]
    batch_max_length = max(batchs_lengths) + 1
    print(batch_max_length)
    inputs_lst, target_lst = [], []

    for item in batch:
        item += [pad_token_id]
        padded = item + [pad_token_id] * (batch_max_length - len(item))
        input = torch.tensor(padded[:-1]) #tensor de entradas
        target = torch.tensor(padded[1:]) #tensor de salidas
        inputs_lst.append(input)
        target_lst.append(target)
    
    inputs_tensor = torch.stack(inputs_lst).to(device)
    target_tensor = torch.stack(target_lst).to(device)
    return inputs_tensor, target_tensor

inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (
    inputs_1,
    inputs_2,
    inputs_3
)
inputs, targets = custom_collate_draft_2(batch)
print(inputs)
print(targets)

6
tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256, 50256, 50256, 50256],
        [    8,     9, 50256, 50256, 50256]])


En  el  siguiente  paso,  se asigna un  valor  de  marcador  de  posición  -100  a  todos  los  tokens  de  relleno. Este  valor  especial   permite  excluir  estos  tokens  de  relleno  de  contribuir  al  cálculo  de  la  pérdida  de  entrenamiento,  garantizando  que  solo  los  datos  significativos  influyan  en  el  aprendizaje  del  modelo.

En la sección 06, no se tuvo que tener en detalle esto, ya que solo se entrenaba el modelo con el último token de salida.

![Texto alternativo](./imgs/7.11.png)

En  el  paso  2.4,  como  se  muestra  en  la  figura, se reemplaza  los  tokens  de  fin  de  texto,  que  previamente se usó  como  tokens  de  relleno  y  cuyo  ID  de  token  es  50256,  por  100  en  la  lista  de  tokens  de  destino.  (La  elección  de  -100  como  reemplazo  se  aclarará  más  adelante).

Sin  embargo,  a tener  en  cuenta  que se conserva  un  token  de  fin  de  texto,  ID  50256,  en  la  lista  de  destinos. Esto  permite  que  el  LLM  aprenda  cuándo  generar  un  token  de  fin  de  texto  en  respuesta  a  las  instrucciones,  lo  cual  se utiliza  como  indicador  de  que  la  respuesta  generada  está  completa.

![Texto alternativo](./imgs/7.12.png)

En  el  siguiente  código,  se modifica  la  función  de  intercalación  personalizada  para  reemplazar  los  tokens  con  ID  50256  por  -100  en  las  listas  de  destino.  Además, se introduce  el  parámetro  "allowed_max_length "  para  limitar  opcionalmente  la  longitud  de  las  muestras.  Este  ajuste  será  útil  si  se planea  trabajar  con  conjuntos  de  datos  propios  que  superen  el  tamaño  de  contexto  de  1024  tokens  compatible  con  el  modelo  GPT2.  El  código  para  esta  función  de  intercalación  actualizada  es  el  siguiente:

In [5]:
def custom_collate_fn(batch, pad_token_id=50256, ignore_index=-100, allowed_max_length=None, device="cpu"):
    batchs_lengths = [len(item) for item in batch]
    batch_max_length = max(batchs_lengths) + 1
    
    inputs_lst, target_lst = [], []

    for item in batch:
        new_item = item.copy()  #copiar para evitar pisar el batch original
        new_item += [pad_token_id]
        padded = new_item + [pad_token_id] * (batch_max_length - len(new_item))
        inputs = torch.tensor(padded[:-1]) #tensor de entradas (truncado)
        targets = torch.tensor(padded[1:]) #tensor de salidas (shift +1)

        mask = targets == pad_token_id  #Reemplazar  todos  los  tokens  de  relleno  excepto  el  primero  en  los  objetivos  por  ignore_index
        indices = torch.nonzero(mask).squeeze()  #devuelve las posiciones (índices) donde mask es True y lo aplana un poco
        if indices.numel() > 1: #si el número de elementos de un tensor es mayor que 1
            targets[indices[1:]] = ignore_index

        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]                  #Truncar  opcionalmente  a  la  longitud  máxima  de  secuencia
            targets = targets[:allowed_max_length]

        inputs_lst.append(inputs)
        target_lst.append(targets)
    
    inputs_tensor = torch.stack(inputs_lst).to(device)
    target_tensor = torch.stack(target_lst).to(device)
    return inputs_tensor, target_tensor

inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]
batch = (
    inputs_1,
    inputs_2,
    inputs_3
)
inputs, targets = custom_collate_fn(batch)
print(inputs)
print(targets)

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256,  -100,  -100,  -100],
        [    8,     9, 50256,  -100,  -100]])


La  función  de  intercalación  modificada  funciona  como  se  espera,  modificando  la  lista  de  destino  insertando  el ID  de  token  -100:

**El proposito de esta modificación es:**

Para  fines  de  demostración,  se considera  el  siguiente  ejemplo  simple  y  autónomo, donde  cada  logit  de  salida  puede  corresponder  a  un  token  potencial  del  vocabulario  del  modelo. Así  es  como se podría  calcular  la  pérdida  de  entropía  cruzada  (introducida  en  la sección 05)  durante el entrenamiento  cuando  el  modelo  predice  una secuencia  de  tokens,  similar  a  lo  que  se ha hecho  en la sección 05  cuando  se  entrena  previamente  el  modelo,  o  en  la sección  06  cuando  se  ajusta  el  modelo  para la clasificación:

In [6]:
logits_1 = torch.tensor(
    [[-1.0,1.0], #predicciones para el primer token
    [-0.5, 1.5]] #predicciones para el segundo token
)

targets_1 = torch.tensor([0, 1])
loss_1 = torch.nn.functional.cross_entropy(logits_1, targets_1)
print(loss_1)

tensor(1.1269)


In [7]:
#Agregar un ID de token adicional, como es de esperar afectará añ cálculo de la pérdida

logits_2 = torch.tensor(
[[-1.0, 1.0],
[-0.5, 1.5],
[-0.5, 1.5]]                                                 #A
)
targets_2 = torch.tensor([0, 1, 1])
loss_2 = torch.nn.functional.cross_entropy(logits_2, targets_2)
print(loss_2)

tensor(0.7936)


Hasta  ahora,  se ha realizado  algunos  cálculos  de  ejemplo  más  o  menos  obvios  utilizando  la  función  de  pérdida  de  entropía  cruzada  en  PyTorch,  la  misma  función  de  pérdida  que  se ha usadao  en  las  funciones  de  entrenamiento  de  las secciones 05 y 06.

Ahora, la  parte  interesante, ver  qué  sucede  si se reemplaza el  tercer  objetivo ID  de  token  con  -100:

In [14]:
targets_3 = torch.tensor([0, 1, -100])
loss_3 = torch.nn.functional.cross_entropy(logits_2, targets_3)
print(loss_3)
print("loss_1 == loss_3:", loss_1 == loss_3)

tensor(1.1269)
loss_1 == loss_3: tensor(True)


Con  base  en  este  resultado,  se puede  observar  que  la  pérdida  resultante  en  estos  tres  ejemplos  de  entrenamiento  es  idéntica  a  la  que calculó  a  partir  de  los  dos  ejemplos  de  entrenamiento  anteriores.  En  otras  palabras,  la  función  de  pérdida  de  entropía  cruzada  ignoró  la  tercera  entrada  del  vector  targets_3 ,  cuyo  ID  de  token  corresponde  a  -100.

Entonces,  **¿qué  tiene  de  especial  100  que  la  pérdida  de  entropía  cruzada  lo  ignora?**  La  configuración  predeterminada  de  la  función  de  entropía  cruzada  en  PyTorch  es  cross_entropy(...,  ignore_index=100).  Esto  significa  que  ignora  los  objetivos  etiquetados  con  -100.

Se va a provechar este ignore_index para ignorar los tokens de relleno (“relleno” que solo está ahí para que todas las secuencias tengan la misma longitud) adicionales.

Sin embargo, se quiere mantener un ID de token 50356 ya que ayuda al LLM a aprender a generar tokens de fin de texto, que se puede utilizar para indicar que una respuesta está completa.
Además de enmascarar los tokens de relleno, también es común enmascarar el objetivo.

![Texto alternativo](./imgs/7.13.png)

Al  ocultar  los  identificadores  de  token  de  destino  correspondientes  a  la  instrucción,  como  se  muestra  en  la  figura ,  el  LLM  calcula  la  pérdida  de  entropía  cruzada  solo  para  los  identificadores  de  destino  de  respuesta  generados.  Al  ocultar  los  tokens  de  instrucción,  el  modelo  se  entrena  para  centrarse  en  generar  respuestas  precisas  en  lugar  de  memorizar  instrucciones,  lo  que  puede  ayudar  a  reducir  el  sobreajuste.

Actualmente,  los  investigadores  no  están  de  acuerdo  sobre  si  enmascarar  las  instrucciones,  como  se  muestra  en  la  figura,  es  beneficioso  durante  el  ajuste  fino  de  instrucciones.

[Creación de cargadores de datos para un cnjunto de datos de instrucciones](./4_cargadores_datos_instrucciones.ipynb)